# Reformat the data files
The data file format and naming changed slightly on 3/22 This script will parse through and make the changes
* Controller only saves when something happens
* 1 file per day, instead of 1 file created on startup
* Humidity and Temperature data are in same file instead of separate ones
* Removed one of the datetime columns, there is no reason to have 2 datetimes (one with and one without timezone)
* Names of files changed slightly

In [1]:
import os
import datetime
import pytz
import pandas as pd


In [2]:
source_data_dir = "/home/neil/growControl_Data/"
migrated_data_dir = "/home/neil/growControl_Data/migrated"

source_data_files = [fname for fname in os.listdir(source_data_dir) if fname[-3:] == "csv"]
source_data_files = sorted(source_data_files)

In [3]:
def create_date(dt):
    '''
    Function to create a date from datetime object for each row in a dataframe
    '''
    return dt.date()



In [4]:
controller_files = [fname for fname in source_data_files if "Controller_ph_pump" in fname]

original_lines = 0
df = None
for fname in controller_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname),index_col="time")
    original_lines += len(data)
    #print("total dispensed: {}".format(data.ph_down_volume.sum()+data.ph_up_volume.sum()))
    if df is None:
        df = data
    else:
        df = df.append(data)
print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df)))
if original_lines != len(df):
    raise ValueError("Some data was not copied from the original format to the new format!")

        
df = df[(df.ph_down_volume!=0.0) | (df.ph_up_volume!=0.0)]
print("Reduced size: {}".format(len(df)))
print("Total volume dispensed: {}".format(df.ph_down_volume.sum()+df.ph_up_volume.sum()))

# Remove the extra datetime column
df = df.drop(axis=1,columns="datetime")

# convert datetime_timezone to a datetime object
df["datetime_timezone"] = pd.to_datetime(df["datetime_timezone"])

# Get the date for each row so we can filter on them
dates = df["datetime_timezone"].apply(create_date)

tzinfo = datetime.datetime.now(pytz.timezone('America/Chicago')).tzinfo
for date in dates.unique():
    start_date = datetime.datetime(year=date.year,month=date.month,day=date.day,tzinfo=tzinfo)
    if start_date < datetime.datetime(year=2020,month=3,day=8,tzinfo=tzinfo):
        start_date = start_date + datetime.timedelta(hours=1)
    end_date = start_date + datetime.timedelta(days=1)
    day_data = df[(df["datetime_timezone"]>=start_date) & (df["datetime_timezone"]<end_date)]
    
    fname = "controller_ph_pump_{:04d}-{:02d}-{:02d}.csv".format(date.year,date.month,date.day)
    with open(os.path.join(migrated_data_dir,fname),'w') as fp:
        day_data.to_csv(fp)

Original lines: 583583 New DataFrame Size: 583583 These must match!
Reduced size: 694
Total volume dispensed: 1658.0


# Sensor_ph Data

In [5]:
ph_files = [fname for fname in source_data_files if fname[:3] == "ph_"]

original_lines = 0
df = None
for fname in ph_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname),index_col="time")
    original_lines += len(data)
    if df is None:
        df = data
    else:
        df = df.append(data)
print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df)))

# Remove the extra datetime column
df = df.drop(axis=1,columns="datetime")

#df = df[df.index<1581745864+1000]

# convert datetime_timezone to a datetime object
df["datetime_timezone"] = pd.to_datetime(df["datetime_timezone"])


# Get the date for each row so we can filter on them
dates = df["datetime_timezone"].apply(create_date)

tzinfo = datetime.datetime.now(pytz.timezone('America/Chicago')).tzinfo

for date in dates.unique():
    start_date = datetime.datetime(year=date.year,month=date.month,day=date.day,tzinfo=tzinfo)
    if start_date < datetime.datetime(year=2020,month=3,day=8,tzinfo=tzinfo):
        start_date = start_date + datetime.timedelta(hours=1)
    end_date = start_date + datetime.timedelta(days=1)
    day_data = df[(df["datetime_timezone"]>=start_date) & (df["datetime_timezone"]<end_date)]
    
    fname = "sensor_ph_bin1_{:04d}-{:02d}-{:02d}.csv".format(date.year,date.month,date.day)
    with open(os.path.join(migrated_data_dir,fname),'w') as fp:
        day_data.to_csv(fp)
print("Done!")

Original lines: 303542 New DataFrame Size: 303542 These must match!
Done!


# Humidity and Temp Data

In [7]:
def round_time(t):
    '''
    Round the time value so it can be merged
    '''
    return round(t,0)

In [13]:
humidity_files = [fname for fname in source_data_files if "humidity_ambient" in fname]
temperature_files = [fname for fname in source_data_files if "temp_ambient" in fname]

original_lines = 0
df = None
for fname in humidity_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname))#,index_col="time")
    data["time"] = data["time"].apply(round_time)
    original_lines += len(data)
    if df is None:
        df = data
    else:
        df = df.append(data)
df = df.set_index("time")
# Remove the extra datetime column
df = df.drop(axis=1,columns="datetime")

print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df)))

original_lines = 0
df_temp = None
for fname in temperature_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname))#,index_col="time")
    data["time"] = data["time"].apply(round_time)
    original_lines += len(data)
    if df_temp is None:
        df_temp = data
    else:
        df_temp = df_temp.append(data)
df_temp = df_temp.set_index("time")

# Remove the duplicate columns from the tempertuare DF
df_temp = df_temp.drop(axis=1,columns="datetime")
df_temp = df_temp.drop(axis=1,columns="datetime_timezone")

print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df_temp)))

df = df.join(df_temp,on="time")
print("Joined size: {}".format(len(df)))
print(df.count())

# convert datetime_timezone to a datetime object
df["datetime_timezone"] = pd.to_datetime(df["datetime_timezone"])

# Get the date for each row so we can filter on them
dates = df["datetime_timezone"].apply(create_date)

tzinfo = datetime.datetime.now(pytz.timezone('America/Chicago')).tzinfo

for date in dates.unique():
    start_date = datetime.datetime(year=date.year,month=date.month,day=date.day,tzinfo=tzinfo)
    if start_date < datetime.datetime(year=2020,month=3,day=8,tzinfo=tzinfo):
        start_date = start_date + datetime.timedelta(hours=1)
    end_date = start_date + datetime.timedelta(days=1)
    day_data = df[(df["datetime_timezone"]>=start_date) & (df["datetime_timezone"]<end_date)]
    
    fname = "humidity_temp_ambient_{:04d}-{:02d}-{:02d}.csv".format(date.year,date.month,date.day)
    with open(os.path.join(migrated_data_dir,fname),'w') as fp:
        day_data.to_csv(fp)
print("Done!")

Original lines: 262084 New DataFrame Size: 262084 These must match!
Original lines: 262086 New DataFrame Size: 262086 These must match!
Joined size: 262084
datetime_timezone            262084
relative_humidity_raw        262084
relative_humidity_average    262084
temperature_raw              252454
temperature_average          252454
dtype: int64
Done!


In [14]:
humidity_files = [fname for fname in source_data_files if "humidity_grow" in fname]
temperature_files = [fname for fname in source_data_files if "temp_grow" in fname]

original_lines = 0
df = None
for fname in humidity_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname))#,index_col="time")
    data["time"] = data["time"].apply(round_time)
    original_lines += len(data)
    if df is None:
        df = data
    else:
        df = df.append(data)
df = df.set_index("time")
# Remove the extra datetime column
df = df.drop(axis=1,columns="datetime")

print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df)))

original_lines = 0
df_temp = None
for fname in temperature_files:
    data = pd.read_csv(os.path.join(source_data_dir,fname))#,index_col="time")
    data["time"] = data["time"].apply(round_time)
    original_lines += len(data)
    if df_temp is None:
        df_temp = data
    else:
        df_temp = df_temp.append(data)
df_temp = df_temp.set_index("time")

# Remove the duplicate columns from the tempertuare DF
df_temp = df_temp.drop(axis=1,columns="datetime")
df_temp = df_temp.drop(axis=1,columns="datetime_timezone")

print("Original lines: {} New DataFrame Size: {} These must match!".format(original_lines,len(df_temp)))

df = df.join(df_temp,on="time")
print("Joined size: {}".format(len(df)))
print(df.count())

# convert datetime_timezone to a datetime object
df["datetime_timezone"] = pd.to_datetime(df["datetime_timezone"])

# Get the date for each row so we can filter on them
dates = df["datetime_timezone"].apply(create_date)

tzinfo = datetime.datetime.now(pytz.timezone('America/Chicago')).tzinfo

for date in dates.unique():
    start_date = datetime.datetime(year=date.year,month=date.month,day=date.day,tzinfo=tzinfo)
    if start_date < datetime.datetime(year=2020,month=3,day=8,tzinfo=tzinfo):
        start_date = start_date + datetime.timedelta(hours=1)
    end_date = start_date + datetime.timedelta(days=1)
    day_data = df[(df["datetime_timezone"]>=start_date) & (df["datetime_timezone"]<end_date)]
    
    fname = "humidity_temp_grow_{:04d}-{:02d}-{:02d}.csv".format(date.year,date.month,date.day)
    with open(os.path.join(migrated_data_dir,fname),'w') as fp:
        day_data.to_csv(fp)
print("Done!")

Original lines: 101689 New DataFrame Size: 101689 These must match!
Original lines: 101689 New DataFrame Size: 101689 These must match!
Joined size: 101689
datetime_timezone            101689
relative_humidity_raw        101689
relative_humidity_average    101689
temperature_raw               97808
temperature_average           97808
dtype: int64
Done!
